# Load data

In [1]:
# Load packages
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import subprocess, time, requests
import ollama
import time
from tqdm import tqdm
import json
import re
import os

In [2]:
# Load data
ds_raw = load_dataset("parquet", data_files="part-0.parquet")["train"]

## Pre subprime crisis

In [3]:
# Filter for year 2006
ds_pre_subprime = ds_raw.filter(lambda x: x["year"] in [2006])

# Add transcript_id as sequential index
ds_pre_subprime = ds_pre_subprime.add_column("transcript_id", list(range(len(ds_pre_subprime))))

# Show first 5 rows
pd.DataFrame(ds_pre_subprime[:5])

,symbol,quarter,year,date,content,structured_content,company_name,company_id,transcript_id
0,A,4,2006,2006-11-14 08:00:00,"Executives: Hilliard C. Terry, Vice President ...","[{'speaker': 'Executives', 'text': 'Hilliard C...","Agilent Technologies, Inc.",154924.0,0
1,A,3,2006,2006-08-14 16:30:00,Executives: Hilliard C. Terry - IR William P. ...,"[{'speaker': 'Executives', 'text': 'Hilliard C...","Agilent Technologies, Inc.",154924.0,1
2,A,2,2006,2006-05-15 16:30:00,"Executives: Hilliard C. Terry - Director, Inve...","[{'speaker': 'Executives', 'text': 'Hilliard C...","Agilent Technologies, Inc.",154924.0,2
3,A,1,2006,2006-02-13 14:45:00,Agilent Technologies Incorporated (NYSE:A) : ...,[{'speaker': 'Agilent Technologies Incorporate...,"Agilent Technologies, Inc.",154924.0,3
4,AAPL,4,2006,2006-10-19 00:15:00,Executives: Peter Oppenheimer - CFO Tim Cook ...,"[{'speaker': 'Executives', 'text': 'Peter Oppe...",Apple Inc.,24937.0,4


In [4]:
ds_pre_subprime

Dataset({
    features: ['symbol', 'quarter', 'year', 'date', 'content', 'structured_content', 'company_name', 'company_id', 'transcript_id'],
    num_rows: 358
})

## Post subprime crisis

In [5]:
# Filter for year 2008
ds_post_subprime = ds_raw.filter(lambda x: x["year"] in [2008])

# Shuffle and select 400 random transcripts. It was aprox. 1200 rows before
ds_post_subprime = ds_post_subprime.shuffle(seed=42).select(range(400))

# Add transcript_id as sequential index
ds_post_subprime = ds_post_subprime.add_column("transcript_id", list(range(len(ds_post_subprime))))

# Show first 5 rows
pd.DataFrame(ds_post_subprime[:5])

,symbol,quarter,year,date,content,structured_content,company_name,company_id,transcript_id
0,APH,4,2008,2009-01-15 13:00:00,Executives: Diana Reardon – SVP & CFO Martin L...,"[{'speaker': 'Executives', 'text': 'Diana Rear...",Amphenol Corporation,24827.0,0
1,PPL,1,2008,2008-05-02 09:00:00,Executives: Timothy Paukovits - Director of IR...,"[{'speaker': 'Executives', 'text': 'Timothy Pa...",None,NaN,1
2,ATO,3,2008,2008-08-06 08:00:00,"Executives: Susan Giles - VP, IR Robert W. Bes...","[{'speaker': 'Executives', 'text': 'Susan Gile...",Atmos Energy Corporation,252684.0,2
3,MDT,4,2008,2008-05-20 08:00:00,"Operator: Good morning. My name is Lucretia, ...","[{'speaker': 'Operator', 'text': 'Good morning...",None,NaN,3
4,PLL,4,2008,2008-09-16 17:00:00,Operator: Welcome to Pall Corporation’s confer...,"[{'speaker': 'Operator', 'text': 'Welcome to P...",None,NaN,4


In [6]:
ds_post_subprime

Dataset({
    features: ['symbol', 'quarter', 'year', 'date', 'content', 'structured_content', 'company_name', 'company_id', 'transcript_id'],
    num_rows: 400
})

In [7]:
# Convert ds_post_subprime to DataFrame (for easy analysis)
df_lengths = pd.DataFrame({
    "transcript_id": [x["transcript_id"] for x in ds_post_subprime],
    "company": [x["company_name"] for x in ds_post_subprime],
    "length_chars": [len(x["content"]) for x in ds_post_subprime]
})

# Basic stats
print("📊 Transcript length summary (characters):")
print(df_lengths["length_chars"].describe())


📊 Transcript length summary (characters):
count       400.000000
mean      55340.840000
std       14951.847407
min          11.000000
25%       47029.750000
50%       53924.500000
75%       62281.750000
max      118215.000000
Name: length_chars, dtype: float64


# Start Ollama

In [8]:
# Define the model name as a constant to be reused
MODEL_NAME = "gemma3:4b"

def start_ollama():
    """Starts Ollama if not already running."""
    try:
        r = requests.get("http://localhost:11434/api/tags", timeout=1)
        if r.status_code == 200:
            print("✅ Ollama is already running.")
            return None
    except requests.exceptions.RequestException:
        pass  # Not running yet
    
    # Start new server
    print("🚀 Starting Ollama server...")
    p = subprocess.Popen(["ollama", "serve"])
    for _ in range(30):
        try:
            r = requests.get("http://localhost:11434/api/tags", timeout=1)
            if r.status_code == 200:
                print("✅ Ollama server is live and ready.")
                return p
        except requests.exceptions.RequestException:
            time.sleep(1)
    raise RuntimeError("Ollama server did not start in time.")

In [9]:
hello_resp = ollama.chat(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": "Say hello in one short sentence."}],
    stream=False
)

print(hello_resp["message"]["content"])

Hello there!


# Ollama

## Cleaning

In [10]:
def extract_first_json(text):
    # remove fences
    text = re.sub(r"```(?:json)?", "", text).strip("` \n")

    # find JSON-like candidates
    candidates = re.findall(r'(\{.*?\}|\[.*?\])', text, re.DOTALL)

    if not candidates:
        print("⚠️ No JSON block found. Raw input:")
        print(text[:2000])
        return []

    def clean(snippet):
        s = snippet
        # remove trailing commas like:  },   ]
        s = re.sub(r",(\s*[}\]])", r"\1", s)
        return s

    for c in candidates:
        cleaned = clean(c)
        try:
            return json.loads(cleaned)
        except Exception as e:
            print("---- Candidate failed ----")
            print("Original candidate:")
            print(c)
            print("---- Cleaned candidate ----")
            print(cleaned)
            print("---- JSON error ----")
            print(e)
            print("---------------------------")
            continue

    print("⚠️ JSON parse failed on all candidates. Raw response:")
    print(text[:2000])
    return []


## Prompt

In [11]:
def extract_risks_from_transcript(text, company_name, transcript_id, model=MODEL_NAME):
    """
    Extract structured risk information from a single transcript text.
    Returns a list of dicts (parsed JSON output).
    """

    prompt = f"""
    Extract all company risk statements from this earnings call transcript.

    Transcript ID: {transcript_id}
    Company: {company_name}

    You must follow these rules exactly:

    - Return ONLY a JSON array. No explanations. No commentary. No prose.
    - Every item must be a JSON object.
    - Use only double quotes for JSON.
    - Do not include speaker names—only their role (e.g., "CEO", "CFO", "Analyst").
    - If expectations tone is unclear, return "neutral".
    - Required fields for each object:
      - "transcript_id"
      - "company"
      - "speaker_role"
      - "risk_type"
      - "expectations"

    Example (structure only):
    [
      {{
        "transcript_id": 123,
        "company": "ExampleCorp",
        "speaker_role": "CEO",
        "risk_type": "operational",
        "expectations": "neutral"
      }}
    ]

    Transcript text:
    \"\"\"{text}\"\"\"
    """



    try:
        response = ollama.chat(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        options={"num_predict": 1000, "temperature": 0}
        )
        content = response["message"]["content"]
        # print("🟢 Raw model output:\n", content[:500])  # debug line
        data = extract_first_json(content)
        # print("🟢 Parsed data:", data) # Debug line

        if isinstance(data, dict):
            data = [data]

        return data

    except Exception as e:
        print(f"⚠️ Error processing transcript {transcript_id} ({company_name}): {e}")
        return []

In [12]:
sample = ds_pre_subprime[0]
result = extract_risks_from_transcript(
    text=sample["content"][:2000],          # use a shortened version for speed
    company_name=sample["company_name"],
    transcript_id=sample["transcript_id"]
)
print(result)

KeyboardInterrupt: 

## Extracting pre subprime crisis

In [ ]:
# --- 1. Where to save ---
output_file = "extracted_risks_2006.csv"
partial_file = "extracted_risks_partial.csv"

# --- 2. Check if there's a partial file to resume ---
if os.path.exists(partial_file):
    df_partial = pd.read_csv(partial_file)
    processed_ids = set(df_partial["transcript_id"].unique())
    all_records = df_partial.to_dict("records")
    print(f"🔁 Resuming from partial file: {len(processed_ids)} transcripts already processed.")
else:
    processed_ids = set()
    all_records = []
    print("🆕 Starting fresh extraction...")

# --- 3. Loop through all transcripts ---
for i, item in enumerate(tqdm(ds_pre_subprime, desc="Extracting risks")):
    transcript_id = item["transcript_id"]

    if transcript_id in processed_ids:
        continue  # already done

    company = item["company_name"]
    text = item["content"][:5000]  # truncate if needed

    records = extract_risks_from_transcript(text, company, transcript_id)
    all_records.extend(records)

    # Save progress every 50 transcripts
    if (i + 1) % 50 == 0:
        pd.DataFrame(all_records).to_csv(partial_file, index=False)
        print(f"💾 Saved partial progress after {i+1} transcripts.")
        time.sleep(1)

# --- 4. Final save ---
df = pd.DataFrame(all_records)
df.to_csv(output_file, index=False)
print(f"✅ Extraction complete! Saved {len(df)} records to {output_file}.")

🆕 Starting fresh extraction...


Extracting risks:  14%|█▎        | 49/358 [39:20<26:55:59, 313.78s/it]

💾 Saved partial progress after 50 transcripts.


Extracting risks:  28%|██▊       | 99/358 [5:06:07<13:16:20, 184.48s/it]

💾 Saved partial progress after 100 transcripts.


...

💾 Saved partial progress after 300 transcripts.


Extracting risks:  97%|█████████▋| 349/358 [8:41:09<03:05, 20.66s/it]

💾 Saved partial progress after 350 transcripts.


Extracting risks: 100%|██████████| 358/358 [8:44:13<00:00, 87.86s/it]

✅ Extraction complete! Saved 1820 records to extracted_risks_2006.csv.


## Extracting post subprime crisis

In [ ]:
# --- 1. Where to save ---
output_file = "extracted_risks_2008.csv"
partial_file = "extracted_risks_partial_2008.csv"

# --- 2. Check if there's a partial file to resume ---
if os.path.exists(partial_file):
    df_partial = pd.read_csv(partial_file)
    processed_ids = set(df_partial["transcript_id"].unique())
    all_records = df_partial.to_dict("records")
    print(f"🔁 Resuming from partial file: {len(processed_ids)} transcripts already processed.")
else:
    processed_ids = set()
    all_records = []
    print("🆕 Starting fresh extraction...")

# --- 3. Loop through all transcripts ---
for i, item in enumerate(tqdm(ds_post_subprime, desc="Extracting risks (2008)")):
    transcript_id = item["transcript_id"]

    if transcript_id in processed_ids:
        continue  # already processed

    company = item["company_name"]
    text = item["content"][:5000]  # truncate for efficiency

    # Run model extraction
    records = extract_risks_from_transcript(text, company, transcript_id)
    all_records.extend(records)

    # Save progress every 50 transcripts
    if (i + 1) % 50 == 0:
        pd.DataFrame(all_records).to_csv(partial_file, index=False)
        print(f"💾 Saved partial progress after {i + 1} transcripts.")
        time.sleep(1)

# --- 4. Final save ---
df = pd.DataFrame(all_records)
df.to_csv(output_file, index=False)
print(f"✅ Extraction complete! Saved {len(df)} records to {output_file}.")


🆕 Starting fresh extraction...


Extracting risks (2008):  12%|█▏        | 49/400 [16:43<1:54:25, 19.56s/it]

💾 Saved partial progress after 50 transcripts.


Extracting risks (2008):  25%|██▍       | 99/400 [35:00<1:39:45, 19.88s/it]

💾 Saved partial progress after 100 transcripts.


...

💾 Saved partial progress after 350 transcripts.


Extracting risks (2008): 100%|█████████▉| 399/400 [2:17:44<00:18, 18.95s/it]

💾 Saved partial progress after 400 transcripts.


Extracting risks (2008): 100%|██████████| 400/400 [2:18:02<00:00, 20.71s/it]

✅ Extraction complete! Saved 2366 records to extracted_risks_2008.csv.
